- Set task name as `nb_set_taskValue` to which this notebook is attached
- You can set in one task and read it from any other task even when it is not linked to that original task where that taskValue set.
  - For example `[nb_set_taskValue => nb_get_taskValue => etc...]` Any notebook in same job which runs post `nb_set_taskValue` can access it.

# Create DF

In [0]:
data = [
    ('Alice', 'Engineering', 1, 100000),
    ('Bob', 'Engineering', 2, 120000),
    ('Charlie', 'HR', 3, 90000),
    ('David', 'Marketing', 4, 110000),
    ('Eve', 'HR', 5, 95000),
    ('Frank', 'Marketing', 6, 105000),
    ('Grace', 'Engineering', 7, 130000),
    ('Heidi', 'HR', 8, 98000)
]
columns = ['employee', 'department', 'level', 'salary']
df = spark.createDataFrame(data, columns)



### SET INT as taskValues

In [0]:
emp_count = df.count()
print(f"Total number of employees: {emp_count}")
dbutils.jobs.taskValues.set("total_emp_count", emp_count)

### Set LIST/SET as taskValue

In [0]:
department_list = [dep.department for dep in df.select('department').collect()]
print('department_list: ', department_list)
dbutils.jobs.taskValues.set("department_list", department_list)

### Set Dictionary as taskValue

In [0]:
from pyspark.sql.functions import *
grouped_df = df.groupBy('department').agg(max(col('salary')).alias('max_salary'))
grouped_dict = {row['department']: row['max_salary'] for row in grouped_df.collect()}
print('grouped_dict: ', grouped_dict)
dbutils.jobs.taskValues.set("max_salary_department", grouped_dict)

### Set it as taskValue and access it as notebook param in receiving end.
- But the notebook should cast the data as dbutils.widgets.text('name','defaultValue','label') will return text as output when reading.
- Need to pass this as `task parameter` and add the taskValue there as input `{{tasks.nb_set_taskValue.values.emp_count_as_nb_param}}` and read it in receiving end using `dbutils.widgets.text` and `dbutils.widgets.text`

In [0]:
emp_count = df.count()
print(f"Total number of employees: {emp_count}")
dbutils.jobs.taskValues.set("emp_count_as_nb_param", emp_count)

### Setting Class as taskValue is not possible
`TypeError: Object of type ABC is not JSON serializable`

In [0]:
# class ABC:
#   def __init__(self, name):
#     self.name = name
    
# abc = ABC('Logesh')
# print('abc.name: ', abc.name)
# dbutils.jobs.taskValues.set("abc", abc)